In [33]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.special import boxcox,boxcox1p, inv_boxcox
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [1]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('4. NSSO68 data set(1).csv')
df.drop(df.columns[0],inplace=True,axis=1)

/home/eshaan/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
df.head()

,grp,Round_Centre,FSU_number,Round,Schedule_Number,Sample,Sector,state,State_Region,District,...,pickle_v,sauce_jam_v,Othrprocessed_v,Beveragestotal_v,foodtotal_v,foodtotal_q,state_1,Region,fruits_df_tt_v,fv_tot
0,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,7,...,0.0,0.0,0.0,0.000000,1141.492400,30.942394,GUJ,2,12.000000,154.18
1,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,7,...,0.0,0.0,0.0,17.500000,1244.553500,29.286153,GUJ,2,333.000000,484.95
2,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,7,...,0.0,0.0,0.0,0.000000,1050.315400,31.527046,GUJ,2,35.000000,214.84
3,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,7,...,0.0,0.0,0.0,33.333333,1142.591667,27.834607,GUJ,2,168.333333,302.30
4,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,7,...,0.0,0.0,0.0,75.000000,945.249500,27.600713,GUJ,2,15.000000,148.00


In [8]:
df.shape

(101662, 383)

In [9]:
df.columns

Index(['grp', 'Round_Centre', 'FSU_number', 'Round', 'Schedule_Number',
       'Sample', 'Sector', 'state', 'State_Region', 'District',
       ...
       'pickle_v', 'sauce_jam_v', 'Othrprocessed_v', 'Beveragestotal_v',
       'foodtotal_v', 'foodtotal_q', 'state_1', 'Region', 'fruits_df_tt_v',
       'fv_tot'],
      dtype='object', length=383)

In [10]:
df.columns.get_loc('MPCE_MRP')

50

In [11]:
df.drop(df.dtypes[df.dtypes == 'object'].index.tolist(),axis=1,inplace=True)

In [12]:
df.shape

(101662, 381)

In [13]:
df.isna().sum()

Round_Centre       0
FSU_number         0
Round              0
Schedule_Number    0
Sample             0
                  ..
foodtotal_v        0
foodtotal_q        0
Region             0
fruits_df_tt_v     0
fv_tot             0
Length: 381, dtype: int64

In [26]:
df.drop(df.isna().sum()[df.isna().sum() > 0].index.tolist(),axis=1,inplace=True)

In [15]:
df.shape

(101662, 345)

In [19]:
x = df.loc[:,df.corr()['MPCE_MRP'][df.corr()['MPCE_MRP']>0.3].index]

In [20]:
x.shape

(101662, 31)

In [21]:
x.drop('MPCE_MRP',axis=1,inplace=True)

In [28]:
(x.isna().sum() > 0)

MPCE_URP          False
milk_q            False
milkprott_q       False
edibletotal_q     False
bananano_q        False
apple_q           False
fruitt_total      False
spicetot_q        False
spicestotal_q     False
pulsestot_v       False
pulsestt_v        False
milk_v            False
Milktotal_v       False
milkprott_v       False
edibletotal_v     False
emftt_v           False
onion_v           False
tamato_v          False
vegtt_v           False
bananano_v        False
apple_v           False
dftt_v            False
sugartotal_v      False
spicetot_v        False
spicestotal_v     False
Biscuits_v        False
foodtotal_v       False
foodtotal_q       False
fruits_df_tt_v    False
fv_tot            False
dtype: bool

In [22]:
df.columns.get_loc('MPCE_MRP')

25

In [30]:
x.drop(x.isna().sum()[x.isna().sum() > 0].index.tolist(),axis=1,inplace=True)

In [31]:
y = df.iloc[:,25]

In [34]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size = 0.8,random_state=100)

In [35]:
model = LinearRegression().fit(x_train,y_train)

In [37]:
pred = model.predict(x_test)

In [38]:
r2_score(y_test,pred)

0.5719314601376304

### Normalization

In [39]:
scaler = StandardScaler()

In [49]:
x_scaled = scaler.fit_transform(x)

In [50]:
x_scaled.shape

(101662, 30)

In [51]:
np.any(np.isnan(x_scaled))

False

In [52]:
np.any(np.isfinite(x_scaled))

True

In [53]:
x_scaled = np.nan_to_num(x_scaled)

In [54]:
x_scaled.shape

(101662, 30)

### Performing PCA

In [55]:
pca = PCA(n_components=0.95)

In [56]:
X = x_scaled - np.mean(x_scaled,axis=0)
cov_mat = np.dot(X.T,X)/X.shape[0]

In [57]:
cov_mat

array([[ 1.        ,  0.17347082,  0.17915311,  0.16897859,  0.19207765,
         0.19250031,  0.26479714,  0.1960377 ,  0.19206244,  0.18271764,
         0.18271764,  0.19453592,  0.22287249,  0.22219512,  0.18612048,
         0.1861996 ,  0.17659906,  0.16478809,  0.19373738,  0.2117967 ,
         0.21403055,  0.16520506,  0.17615837,  0.22255855,  0.21616589,
         0.15593969,  0.31560134,  0.19657759,  0.27145099,  0.28595461],
       [ 0.17347082,  1.        ,  0.99851482,  0.29511031,  0.1534594 ,
         0.27653781,  0.2857595 ,  0.18777648,  0.19218955,  0.29161606,
         0.29161606,  0.95321303,  0.90731132,  0.91010455,  0.28050735,
        -0.06069733,  0.23944036,  0.24515985,  0.18278401,  0.19340609,
         0.24648217,  0.22401215,  0.44614384,  0.25391882,  0.26273137,
         0.11249616,  0.59508268,  0.72793225,  0.3182997 ,  0.30791477],
       [ 0.17915311,  0.99851482,  1.        ,  0.30564966,  0.16061786,
         0.28327166,  0.29529465,  0.1961324 ,  0

In [59]:
x = pca.fit_transform(x_scaled)

In [60]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size = 0.8,random_state=100)

In [61]:
model_pca = LinearRegression().fit(x_train,y_train)

In [62]:
pred_pca = model_pca.predict(x_test)

In [63]:
r2_score(y_test,pred_pca)

0.5605539090335274

In [64]:
pca.explained_variance_ratio_

array([0.40007658, 0.12684011, 0.09379072, 0.05502317, 0.04635376,
       0.04392408, 0.03530601, 0.03085276, 0.02787466, 0.02756175,
       0.02355136, 0.01656072, 0.01535452, 0.01440669])